In [2]:
import pandas as pd
import json

import nltk
import pandas as pd
import numpy as np
import re
import time
import sys
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from pprint import pprint
from functools import reduce
from matplotlib import pyplot as plt
import string

import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import KFold
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
from tqdm import tqdm
from tabulate import tabulate
import argparse

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

C:\Users\navvu\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
temp = pd.read_csv('For_preprocessing.csv')
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4768 entries, 0 to 4767
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review_Text  4763 non-null   object
 1   Sentiment    4766 non-null   object
dtypes: object(2)
memory usage: 74.6+ KB


In [4]:
temp

,Review_Text,Sentiment
0,Its really nice place to stay especially for b...,3
1,It seems that hotel does not check the basic a...,1
2,Worst hotel I have ever encountered. I will ne...,1
3,Had a good time in this hotel and the staff Ku...,3
4,good hotel and staff Veg food good non veg bre...,3
...,...,...
4763,My fifth stay at the hotel for business. Rooms...,3
4764,enjoyable,3
4765,Most impressive service by staff in all areas....,3
4766,"The linens were smelling bad, and the elevator...",1


In [5]:
temp=temp.dropna(subset=['Review_Text'])
temp=temp.dropna(subset=['Sentiment'])
temp = temp.reset_index(drop=True)
temp_val = temp['Sentiment'].tolist()
#len(temp_val)
ind_list = []

for i in range(len(temp['Sentiment'])):
    if temp_val[i] != '1' and temp_val[i] != '2' and temp_val[i] != '3':
        ind_list.append(i)
temp.reset_index(drop=True)
temp.drop(temp.index[ind_list], inplace=True)

In [6]:
type(temp['Sentiment'][0])

str

In [7]:
temp = temp.reset_index(drop=True)
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4699 entries, 0 to 4698
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review_Text  4699 non-null   object
 1   Sentiment    4699 non-null   object
dtypes: object(2)
memory usage: 73.5+ KB


In [59]:
# def text_processing(tweet):
    
#     #Generating the list of words in the tweet (punctuations removed)
#     def form_sentence(tweet):
#         tweet_blob = TextBlob(tweet)
#         return ' '.join(tweet_blob.words)
#     new_tweet = form_sentence(tweet)
    
#     #Removing stopwords and words with unusual symbols
#     def no_user_alpha(tweet):
#         tweet_list = [ele for ele in tweet.split()]
#         clean_tokens = [t for t in tweet_list if re.match(r'[^\W\d]*$', t)]
#         clean_s = ' '.join(clean_tokens)
#         clean_mess = [word for word in clean_s.split() if word.lower() not in stopwords.words('english')]
#         return clean_mess
#     no_punc_tweet = no_user_alpha(new_tweet)
    
#     #Normalizing the words in tweets 
#     def normalization(tweet_list):
#         lem = WordNetLemmatizer()
#         normalized_tweet = []
#         for word in tweet_list:
#             normalized_text = lem.lemmatize(word,'v')
#             normalized_tweet.append(normalized_text)
#         return normalized_tweet
    
    
#     return normalization(no_punc_tweet)

In [8]:
def tokenize_reg():
    temp['Review_Text'] = temp['Review_Text'].replace(r'http\S+','',regex=True).replace(r'www\S+','',regex=True).replace(r'\d+','',regex=True)
    tokens = RegexpTokenizer(r'\w+')
    temp['Review_Text']=temp['Review_Text'].apply(lambda x:tokens.tokenize(x.lower()))
    stop_words = set(stopwords.words('english')) 
    stop_words = stop_words.union(",","(",")","[","]","{","}","#","@","!",":",";",".","?")
    temp['Review_Text'] = temp['Review_Text'].apply(lambda x: [item for item in x if item not in stop_words])
tokenize_reg()

In [9]:
lem = WordNetLemmatizer()
def lemm(text):
    sent=[]
    for word in text:
        sent.append(lem.lemmatize(word))
    return sent

ps=PorterStemmer()

def stemm(text):        
    sent = []
    for word in text:
        sent.append(ps.stem(word))
    return sent

temp['Review_Text'] =  temp.apply(lambda x: stemm(lemm(x['Review_Text'])), axis=1)

In [10]:
revs=temp['Review_Text'].tolist()
for i in range(len(revs)):
    revs[i]=' '.join(revs[i])
temp['Review_Text'] = revs
temp

,Review_Text,Sentiment
0,realli nice place stay especi busi tourist purpos,3
1,seem hotel check basic amen room hand room tra...,1
2,worst hotel ever encount never think stay thii...,1
3,good time hotel staff kumar aishwarya hous kee...,3
4,good hotel staff veg food good non veg breakfa...,3
...,...,...
4694,fifth stay hotel busi room great restaur excel...,3
4695,enjoy,3
4696,impress servic staff area good restaur fit cen...,3
4697,linen smell bad elev pungent odour housekeep p...,1


In [11]:
st = temp['Sentiment'].tolist()
st=set(st)

In [12]:
st

{'1', '2', '3'}

In [14]:
X = temp['Review_Text']
y = temp['Sentiment']

#test = test_tweets['post']
from sklearn.model_selection import train_test_split
msg_train, msg_test, label_train, label_test = train_test_split(temp['Review_Text'], temp['Sentiment'], test_size=0.2)
#Machine Learning Pipeline

pipeline = Pipeline([
    ('bow',CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])
pipeline.fit(msg_train,label_train)

tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}



In [20]:
from sklearn.metrics import classification_report

score = 'f1_macro'
print("# Tuning hyper-parameters for %s" % score)
print()
np.errstate(divide='ignore')
clf = GridSearchCV(pipeline, tuned_parameters, cv=10, scoring=score)
clf.fit(msg_train, msg_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
for mean, std, params in zip(clf.cv_results_['mean_test_score'], 
                             clf.cv_results_['std_test_score'], 
                             clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print(classification_report(y_test, clf.predict(x_test), digits=4))
print()

# Tuning hyper-parameters for f1_macro



C:\Users\navvu\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


ValueError: Invalid parameter clf for estimator Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())]). Check the list of available parameters with `estimator.get_params().keys()`.

In [17]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(pipeline, tuned_parameters, cv=10, scoring=score)
clf.fit(x_train, y_train)

print(classification_report(y_test, clf.predict(x_test), digits=4))

NameError: name 'score' is not defined

In [64]:
predictions = pipeline.predict(msg_test)

# print("Classification report")
# print(classification_report(predictions,label_test))
print ("Confusion Matrix")
print(confusion_matrix(predictions,label_test))
print("Accuracy Score")
print(accuracy_score(predictions,label_test))

Confusion Matrix
[[  3   0   0]
 [  0  14   1]
 [ 94 144 684]]
Accuracy Score
0.7457446808510638


In [65]:
print("Classification report")
print(classification_report(predictions,label_test))

Classification report
              precision    recall  f1-score   support

           1       0.03      1.00      0.06         3
           2       0.09      0.93      0.16        15
           3       1.00      0.74      0.85       922

    accuracy                           0.75       940
   macro avg       0.37      0.89      0.36       940
weighted avg       0.98      0.75      0.84       940



In [66]:
df = pd.read_csv('kolkata_reviews.csv',encoding='utf-8')
df

,text
0,I've made a booking with the help of Mr. Manoj...
1,I have been to this property in Jan 2022. Very...
2,"Very good stay , loved the hospitality and war..."
3,I had booked Holiday Inn express for 2 days fo...
4,Excellent stay.. well maintained property.. ve...
...,...
1995,During kolkata office visit stayed at Golden t...
1996,I was at the Golden Tulip for 5 days. And it w...
1997,The hotel does not have a very prominent locat...
1998,"I had booked executive room, but the room prov..."


In [67]:
predictions = pipeline.predict(df['text'])

In [68]:
df['Sentiment']=predictions
df

,text,Sentiment
0,I've made a booking with the help of Mr. Manoj...,3
1,I have been to this property in Jan 2022. Very...,3
2,"Very good stay , loved the hospitality and war...",3
3,I had booked Holiday Inn express for 2 days fo...,3
4,Excellent stay.. well maintained property.. ve...,3
...,...,...
1995,During kolkata office visit stayed at Golden t...,3
1996,I was at the Golden Tulip for 5 days. And it w...,3
1997,The hotel does not have a very prominent locat...,3
1998,"I had booked executive room, but the room prov...",3


In [69]:
df['Sentiment'].value_counts()

3    2000
Name: Sentiment, dtype: int64